#### This code file is same as of 1 ANN training using iterative feature selection process with base model being on common features.

#### Changes made are in this file considers all five output columns for generating a final output binary value. Other hyperparameters too ar changed

In [15]:
#Import all the necessary libraries
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as nm
nm.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
import time


path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename1='Common_Data.xlsx'
filename2='Intersection_Data.xlsx'

#Access all the necessary variables and datasets
%store -r common_data 
%store -r random_forest_data
random_forest_data.shape #161 statistical features and one output column
# Using the common_data dataframe created in ANN_Model_on_common_features
#Using the appended_data dataframe created in Random_Forest_Code 
common_data

GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


,NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN,MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN,STEELLEVELINMOLD-SETMEAN,SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN,SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN,LIQUIDUSTEMPMEAN,NARROWFACE1WATERFLOWMEAN,TUNDISHWEIGHTMEAN,OUTPUT
0,4.925862,4.000000,80.103621,1.177759,190.958678,1521.40790,461.735632,23.854885,1
1,4.915455,4.000000,80.104899,1.170657,195.140556,1521.76000,461.954545,23.548081,1
2,5.482414,4.000000,80.106667,1.239828,191.464138,1521.26000,461.534483,24.320115,0
3,5.471203,4.000000,80.108861,1.240000,187.756139,1521.74000,461.892405,24.275696,0
4,5.081299,4.000000,80.110282,1.190000,188.373955,1521.41000,462.028249,24.563785,1
...,...,...,...,...,...,...,...,...,...
586,2.140476,7.994048,80.108333,0.789524,213.972321,1514.56430,452.720238,23.541369,0
587,2.098624,8.052910,80.107513,0.334974,204.833228,1515.94260,443.820106,27.779471,1
588,2.097216,7.407216,80.109175,0.544330,213.607887,1512.56210,452.283505,22.814948,1
589,2.091311,7.121359,80.106019,0.722087,31.653689,1512.88320,452.150485,24.742087,1


#### Training the base model which is done in six steps 1) Splitting the data 2)Scaling the data 3)Defining the model 4)Compiling the model 5)Fit the data  6) Evaluate the model.

In [16]:
common_data = shuffle(common_data, random_state=42)
#Defining train and test sizes
total_length= common_data.shape[0]
train_size=int(common_data.shape[0]*0.8)
test_size=common_data.shape[0]-train_size

# First converting dataframe series into numpy array and setting the datatype as float32 for all data
# In second step converting all 1D array into 2D array
train_X=common_data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

test_X=common_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

print('Dimension of training data',train_X.shape)
print('Dimension of testing data',test_X.shape)

#print(train_X)
print(type(train_X))
print(train_Y.shape)
test_Y.shape #2-D array
#train_Y

Dimension of training data (472, 8)
Dimension of testing data (119, 8)
<class 'numpy.ndarray'>
(472, 1)


(119, 1)

In [17]:
#scaler=StandardScaler()
#scaler=MinMaxScaler(feature_range=(0,1)) # scaler object
scaler=RobustScaler()
# First fit the scaler with your training data.
# It will find those parameters using which it is going to scale.
# Then transform any data you want.

train_X=scaler.fit_transform(train_X) 

# Since it has scaled  the data as per the parameters it learnt during training data.
# So it will require 2D array/similiar dimension as of training data.
# So testing data should be a 2D array.
# Testing data only need to be transformed on the basis of the parameters learnt using fit function on training data.

test_X=scaler.transform(test_X) #Scaling the training data and using the same parameters for testing data
#test
#train_X[:]

In [18]:

#Creating a Model
print('Total No of features',len(train_X[0,:])) #No of features
base_model=Sequential() 

base_model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
base_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
base_model.summary()

Total No of features 8
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 50)                450       
                                                                 
 dense_31 (Dense)            (None, 1)                 51        
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


In [19]:
#model.compile(optimizer='_' , loss='_' , metrics=['accuracy'])
base_model.compile(loss='binary_crossentropy',optimizer=SGD(momentum=0.9),metrics=['accuracy'])

In [20]:
#Fitting the training data
#base_model.fit(training data input, training data output, batch_size=_,validation_data=(x_test,y_test))
base_model.fit(train_X,train_Y,epochs=150,batch_size=len(train_X),validation_data=(test_X,test_Y))

# Epochs is number of times i want to do forward and backward propogation.
# batch size is used for update the weights in mini batches. The default back size is 32.
# Validation data : accuracy looking like on validation data after every epoch, you can give test data here too. 
#The loss will be reduced due to backpropgation through time.

#Add regularization or increase number of epochs for better accuracy.

Epoch 1/150
1/1 [==============================] - 0s 471ms/step - loss: 20.4979 - accuracy: 0.5233 - val_loss: 6.6152 - val_accuracy: 0.5630
Epoch 2/150
1/1 [==============================] - 0s 47ms/step - loss: 10.5079 - accuracy: 0.5678 - val_loss: 8.1061 - val_accuracy: 0.5798
Epoch 3/150
1/1 [==============================] - 0s 47ms/step - loss: 13.0254 - accuracy: 0.5699 - val_loss: 4.2342 - val_accuracy: 0.5798
Epoch 4/150
1/1 [==============================] - 0s 63ms/step - loss: 6.5092 - accuracy: 0.5869 - val_loss: 1.0057 - val_accuracy: 0.5210
Epoch 5/150
1/1 [==============================] - 0s 63ms/step - loss: 1.0625 - accuracy: 0.5551 - val_loss: 0.9062 - val_accuracy: 0.5126
Epoch 6/150
1/1 [==============================] - 0s 47ms/step - loss: 0.8559 - accuracy: 0.5572 - val_loss: 0.9365 - val_accuracy: 0.5126
Epoch 7/150
1/1 [==============================] - 0s 47ms/step - loss: 0.8724 - accuracy: 0.5487 - val_loss: 0.9754 - val_accuracy: 0.5042
Epoch 8/150
1/1 

1/1 [==============================] - 0s 51ms/step - loss: 0.9794 - accuracy: 0.6335 - val_loss: 1.0416 - val_accuracy: 0.5546
Epoch 60/150
1/1 [==============================] - 0s 43ms/step - loss: 0.9755 - accuracy: 0.6314 - val_loss: 1.0384 - val_accuracy: 0.5546
Epoch 61/150
1/1 [==============================] - 0s 31ms/step - loss: 0.9719 - accuracy: 0.6398 - val_loss: 1.0351 - val_accuracy: 0.5546
Epoch 62/150
1/1 [==============================] - 0s 31ms/step - loss: 0.9683 - accuracy: 0.6356 - val_loss: 1.0318 - val_accuracy: 0.5630
Epoch 63/150
1/1 [==============================] - 0s 47ms/step - loss: 0.9649 - accuracy: 0.6377 - val_loss: 1.0285 - val_accuracy: 0.5630
Epoch 64/150
1/1 [==============================] - 0s 32ms/step - loss: 0.9616 - accuracy: 0.6462 - val_loss: 1.0250 - val_accuracy: 0.5630
Epoch 65/150
1/1 [==============================] - 0s 46ms/step - loss: 0.9584 - accuracy: 0.6441 - val_loss: 1.0215 - val_accuracy: 0.5546
Epoch 66/150
1/1 [========

1/1 [==============================] - 0s 32ms/step - loss: 0.8603 - accuracy: 0.7013 - val_loss: 0.9153 - val_accuracy: 0.6891
Epoch 118/150
1/1 [==============================] - 0s 31ms/step - loss: 0.8590 - accuracy: 0.7055 - val_loss: 0.9139 - val_accuracy: 0.6891
Epoch 119/150
1/1 [==============================] - 0s 47ms/step - loss: 0.8577 - accuracy: 0.7076 - val_loss: 0.9126 - val_accuracy: 0.6891
Epoch 120/150
1/1 [==============================] - 0s 29ms/step - loss: 0.8565 - accuracy: 0.7119 - val_loss: 0.9113 - val_accuracy: 0.6975
Epoch 121/150
1/1 [==============================] - 0s 47ms/step - loss: 0.8552 - accuracy: 0.7161 - val_loss: 0.9099 - val_accuracy: 0.6975
Epoch 122/150
1/1 [==============================] - 0s 32ms/step - loss: 0.8540 - accuracy: 0.7182 - val_loss: 0.9086 - val_accuracy: 0.7059
Epoch 123/150
1/1 [==============================] - 0s 31ms/step - loss: 0.8527 - accuracy: 0.7182 - val_loss: 0.9073 - val_accuracy: 0.7059
Epoch 124/150
1/1 [=

In [21]:
# Create a list to store the selected features and their corresponding accuracies
base_features = common_data.columns.tolist()[:-1]
base_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'LIQUIDUSTEMPMEAN',
 'NARROWFACE1WATERFLOWMEAN',
 'TUNDISHWEIGHTMEAN']

In [22]:
# Evaluate the base model

testPredict_probs=base_model.predict(test_X)
#testPredict_probs

# Apply threshold to obtain predicted classes
threshold = 0.5  # Adjust this threshold as needed
y_pred = (testPredict_probs > threshold).astype(int)
y_pred

base_accuracy = accuracy_score(test_Y,y_pred)
base_accuracy

4/4 [==============================] - 0s 0s/step


0.7142857142857143

### Data for new model

In [23]:
%store -r feature_imp_RF 
#using it from Random_Forest_Code
#it is a dataframe which stores the features and their corresponding importance score in descending order
total_features=feature_imp_RF["Feature"]
total_features_list= total_features.tolist()

remaining_features=[]
for feature in total_features_list:
    if feature not in base_features:
        remaining_features.append(feature) 
        
print(len(remaining_features))
#feature_imp_RF["Feature"]

153


### Iterating over remaining features and training at each iteration

In [24]:
new_features=[]
optimal_features=[]
accuracy_store=[base_accuracy]

# Start the timer
start_time = time.time()

for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]
    
    #Creating a new data and shuffling the data
    new_data=random_forest_data.loc[:,new_features]
    new_data = shuffle(new_data, random_state=42)
    
    #1 Splitting the new_data into train and test
    new_total_length= new_data.shape[0]
    new_train_size=int(new_data.shape[0]*0.8)
    new_test_size=new_data.shape[0]-new_train_size
    
    new_train_X=new_data.iloc[:,:][0:new_train_size].to_numpy().astype(nm.float32) #2D array
    new_train_Y=random_forest_data['OUTPUT'][0:new_train_size].to_numpy().astype(nm.float32) #1D array
    new_train_Y=nm.reshape(new_train_Y,(new_train_Y.shape[0],1)) # 2D array
    
    new_test_X=new_data.iloc[:,:][new_train_size:].to_numpy().astype(nm.float32) #2D array
    new_test_Y=random_forest_data['OUTPUT'][new_train_size:].to_numpy().astype(nm.float32) #1D array
    new_test_Y=nm.reshape(new_test_Y,(new_test_Y.shape[0],1)) # 2D array}

    #2 Scaling the data
    scaler=RobustScaler()
    new_train_X=scaler.fit_transform(new_train_X) 
    new_test_X=scaler.transform(new_test_X)
    
    #3 Create a new model with the updated feature set
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
    #4 Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer=SGD(momentum=0.9),metrics=['accuracy'])
    
    print(feature)
    
    #5 Train the new model
    new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=len(new_train_X),validation_data=(new_test_X,new_test_Y))
    
    #6 Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    accuracy_store.append(new_accuracy)
    
    # Update the baseline feature set if the accuracy score improved
    if new_accuracy> base_accuracy:
        base_features = new_features
        base_accuracy = new_accuracy
        optimal_features.append(new_features)
    
    # Check for convergence
    #if len(base_features) == random_forest_data.iloc[:,:-1].shape[1]:
     #   break
        
# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

MOLDWIDTHTOPMEAN
Epoch 1/150
1/1 [==============================] - 1s 564ms/step - loss: 3.5689 - accuracy: 0.5085 - val_loss: 29.0620 - val_accuracy: 0.5210
Epoch 2/150
1/1 [==============================] - 0s 47ms/step - loss: 28.4221 - accuracy: 0.4788 - val_loss: 8.3831 - val_accuracy: 0.5462
Epoch 3/150
1/1 [==============================] - 0s 47ms/step - loss: 8.2264 - accuracy: 0.4788 - val_loss: 4.3110 - val_accuracy: 0.5798
Epoch 4/150
1/1 [==============================] - 0s 62ms/step - loss: 18.3729 - accuracy: 0.4915 - val_loss: 4.2147 - val_accuracy: 0.5798
Epoch 5/150
1/1 [==============================] - 0s 44ms/step - loss: 17.6869 - accuracy: 0.4915 - val_loss: 2.1257 - val_accuracy: 0.5798
Epoch 6/150
1/1 [==============================] - 0s 56ms/step - loss: 7.0095 - accuracy: 0.4915 - val_loss: 2.5573 - val_accuracy: 0.5210
Epoch 7/150
1/1 [==============================] - 0s 42ms/step - loss: 2.5883 - accuracy: 0.4682 - val_loss: 2.5764 - val_accuracy: 0.512

1/1 [==============================] - 0s 63ms/step - loss: 1.3310 - accuracy: 0.4936 - val_loss: 1.3588 - val_accuracy: 0.5210
Epoch 60/150
1/1 [==============================] - 0s 63ms/step - loss: 1.3292 - accuracy: 0.4915 - val_loss: 1.3579 - val_accuracy: 0.5210
Epoch 61/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3267 - accuracy: 0.4958 - val_loss: 1.3549 - val_accuracy: 0.5210
Epoch 62/150
1/1 [==============================] - 0s 32ms/step - loss: 1.3233 - accuracy: 0.4915 - val_loss: 1.3499 - val_accuracy: 0.5042
Epoch 63/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3190 - accuracy: 0.5000 - val_loss: 1.3433 - val_accuracy: 0.5210
Epoch 64/150
1/1 [==============================] - 0s 63ms/step - loss: 1.3139 - accuracy: 0.4979 - val_loss: 1.3357 - val_accuracy: 0.5042
Epoch 65/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3085 - accuracy: 0.5000 - val_loss: 1.3278 - val_accuracy: 0.5294
Epoch 66/150
1/1 [========

1/1 [==============================] - 0s 37ms/step - loss: 1.1672 - accuracy: 0.5826 - val_loss: 1.1939 - val_accuracy: 0.5966
Epoch 118/150
1/1 [==============================] - 0s 42ms/step - loss: 1.1651 - accuracy: 0.5826 - val_loss: 1.1917 - val_accuracy: 0.5966
Epoch 119/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1629 - accuracy: 0.5805 - val_loss: 1.1894 - val_accuracy: 0.5966
Epoch 120/150
1/1 [==============================] - 0s 39ms/step - loss: 1.1608 - accuracy: 0.5784 - val_loss: 1.1873 - val_accuracy: 0.5966
Epoch 121/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1587 - accuracy: 0.5763 - val_loss: 1.1851 - val_accuracy: 0.5966
Epoch 122/150
1/1 [==============================] - 0s 47ms/step - loss: 1.1566 - accuracy: 0.5763 - val_loss: 1.1830 - val_accuracy: 0.5966
Epoch 123/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1546 - accuracy: 0.5805 - val_loss: 1.1809 - val_accuracy: 0.5966
Epoch 124/150
1/1 [=

1/1 [==============================] - 0s 32ms/step - loss: 1.6772 - accuracy: 0.5127 - val_loss: 1.3491 - val_accuracy: 0.5630
Epoch 25/150
1/1 [==============================] - 0s 47ms/step - loss: 1.4632 - accuracy: 0.5064 - val_loss: 2.2498 - val_accuracy: 0.5378
Epoch 26/150
1/1 [==============================] - 0s 47ms/step - loss: 1.5876 - accuracy: 0.5275 - val_loss: 2.5856 - val_accuracy: 0.5378
Epoch 27/150
1/1 [==============================] - 0s 31ms/step - loss: 1.6682 - accuracy: 0.5360 - val_loss: 2.4401 - val_accuracy: 0.5630
Epoch 28/150
1/1 [==============================] - 0s 31ms/step - loss: 1.6159 - accuracy: 0.5360 - val_loss: 2.0012 - val_accuracy: 0.5798
Epoch 29/150
1/1 [==============================] - 0s 48ms/step - loss: 1.4858 - accuracy: 0.5445 - val_loss: 1.3944 - val_accuracy: 0.5630
Epoch 30/150
1/1 [==============================] - 0s 32ms/step - loss: 1.3220 - accuracy: 0.5360 - val_loss: 1.2592 - val_accuracy: 0.5546
Epoch 31/150
1/1 [========

1/1 [==============================] - 0s 47ms/step - loss: 1.0470 - accuracy: 0.5720 - val_loss: 1.0770 - val_accuracy: 0.5126
Epoch 83/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0450 - accuracy: 0.5720 - val_loss: 1.0754 - val_accuracy: 0.5042
Epoch 84/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0430 - accuracy: 0.5699 - val_loss: 1.0739 - val_accuracy: 0.5042
Epoch 85/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0410 - accuracy: 0.5763 - val_loss: 1.0726 - val_accuracy: 0.5042
Epoch 86/150
1/1 [==============================] - 0s 53ms/step - loss: 1.0391 - accuracy: 0.5678 - val_loss: 1.0715 - val_accuracy: 0.5126
Epoch 87/150
1/1 [==============================] - 0s 41ms/step - loss: 1.0371 - accuracy: 0.5657 - val_loss: 1.0704 - val_accuracy: 0.5042
Epoch 88/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0353 - accuracy: 0.5657 - val_loss: 1.0692 - val_accuracy: 0.4958
Epoch 89/150
1/1 [========

...

1/1 [==============================] - 0s 31ms/step - loss: 1.2472 - accuracy: 0.5254 - val_loss: 1.2778 - val_accuracy: 0.4790
Epoch 56/150
1/1 [==============================] - 0s 31ms/step - loss: 1.2443 - accuracy: 0.5381 - val_loss: 1.2804 - val_accuracy: 0.4874
Epoch 57/150
1/1 [==============================] - 0s 33ms/step - loss: 1.2428 - accuracy: 0.5445 - val_loss: 1.2823 - val_accuracy: 0.4958
Epoch 58/150
1/1 [==============================] - 0s 47ms/step - loss: 1.2419 - accuracy: 0.5424 - val_loss: 1.2830 - val_accuracy: 0.4790
Epoch 59/150
1/1 [==============================] - 0s 31ms/step - loss: 1.2408 - accuracy: 0.5339 - val_loss: 1.2820 - val_accuracy: 0.4790
Epoch 60/150
1/1 [==============================] - 0s 32ms/step - loss: 1.2391 - accuracy: 0.5318 - val_loss: 1.2793 - val_accuracy: 0.4706
Epoch 61/150
1/1 [==============================] - 0s 32ms/step - loss: 1.2364 - accuracy: 0.5318 - val_loss: 1.2748 - val_accuracy: 0.4706
Epoch 62/150
1/1 [========

1/1 [==============================] - 0s 30ms/step - loss: 1.1055 - accuracy: 0.6102 - val_loss: 1.1285 - val_accuracy: 0.5798
Epoch 114/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1036 - accuracy: 0.6081 - val_loss: 1.1266 - val_accuracy: 0.5798
Epoch 115/150
1/1 [==============================] - 0s 32ms/step - loss: 1.1017 - accuracy: 0.6081 - val_loss: 1.1247 - val_accuracy: 0.5714
Epoch 116/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0998 - accuracy: 0.6038 - val_loss: 1.1228 - val_accuracy: 0.5630
Epoch 117/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0979 - accuracy: 0.6081 - val_loss: 1.1209 - val_accuracy: 0.5630
Epoch 118/150
1/1 [==============================] - 0s 32ms/step - loss: 1.0961 - accuracy: 0.6081 - val_loss: 1.1190 - val_accuracy: 0.5630
Epoch 119/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0942 - accuracy: 0.6102 - val_loss: 1.1170 - val_accuracy: 0.5630
Epoch 120/150
1/1 [=

1/1 [==============================] - 0s 31ms/step - loss: 1.8144 - accuracy: 0.5148 - val_loss: 1.7106 - val_accuracy: 0.4706
Epoch 21/150
1/1 [==============================] - 0s 32ms/step - loss: 1.7919 - accuracy: 0.5000 - val_loss: 1.7043 - val_accuracy: 0.4706
Epoch 22/150
1/1 [==============================] - 0s 31ms/step - loss: 1.7575 - accuracy: 0.5021 - val_loss: 1.6794 - val_accuracy: 0.4706
Epoch 23/150
1/1 [==============================] - 0s 46ms/step - loss: 1.7143 - accuracy: 0.4958 - val_loss: 1.6401 - val_accuracy: 0.4706
Epoch 24/150
1/1 [==============================] - 0s 32ms/step - loss: 1.6647 - accuracy: 0.4979 - val_loss: 1.5904 - val_accuracy: 0.4790
Epoch 25/150
1/1 [==============================] - 0s 31ms/step - loss: 1.6104 - accuracy: 0.4979 - val_loss: 1.5340 - val_accuracy: 0.4874
Epoch 26/150
1/1 [==============================] - 0s 31ms/step - loss: 1.5537 - accuracy: 0.5021 - val_loss: 1.4763 - val_accuracy: 0.4958
Epoch 27/150
1/1 [========

1/1 [==============================] - 0s 31ms/step - loss: 1.2081 - accuracy: 0.5975 - val_loss: 1.2373 - val_accuracy: 0.5798
Epoch 79/150
1/1 [==============================] - 0s 22ms/step - loss: 1.2059 - accuracy: 0.6017 - val_loss: 1.2362 - val_accuracy: 0.5798
Epoch 80/150
1/1 [==============================] - 0s 31ms/step - loss: 1.2037 - accuracy: 0.5996 - val_loss: 1.2347 - val_accuracy: 0.5882
Epoch 81/150
1/1 [==============================] - 0s 31ms/step - loss: 1.2015 - accuracy: 0.5953 - val_loss: 1.2327 - val_accuracy: 0.5882
Epoch 82/150
1/1 [==============================] - 0s 32ms/step - loss: 1.1993 - accuracy: 0.5953 - val_loss: 1.2301 - val_accuracy: 0.5882
Epoch 83/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1969 - accuracy: 0.5975 - val_loss: 1.2269 - val_accuracy: 0.5882
Epoch 84/150
1/1 [==============================] - 0s 31ms/step - loss: 1.1944 - accuracy: 0.5975 - val_loss: 1.2234 - val_accuracy: 0.5798
Epoch 85/150
1/1 [========

Epoch 136/150
1/1 [==============================] - 0s 32ms/step - loss: 1.0887 - accuracy: 0.6377 - val_loss: 1.1145 - val_accuracy: 0.6050
Epoch 137/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0870 - accuracy: 0.6356 - val_loss: 1.1127 - val_accuracy: 0.6050
Epoch 138/150
1/1 [==============================] - 0s 50ms/step - loss: 1.0853 - accuracy: 0.6356 - val_loss: 1.1109 - val_accuracy: 0.6050
Epoch 139/150
1/1 [==============================] - 0s 28ms/step - loss: 1.0835 - accuracy: 0.6335 - val_loss: 1.1092 - val_accuracy: 0.6050
Epoch 140/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0818 - accuracy: 0.6314 - val_loss: 1.1076 - val_accuracy: 0.6050
Epoch 141/150
1/1 [==============================] - 0s 41ms/step - loss: 1.0801 - accuracy: 0.6292 - val_loss: 1.1059 - val_accuracy: 0.6050
Epoch 142/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0784 - accuracy: 0.6292 - val_loss: 1.1043 - val_accuracy: 0.6050
Epoch 

In [25]:
print("Simulation ended",training_time)
print("Best feature set:", base_features)
print("Best accuracy score:", base_accuracy)

Simulation ended 1092.7251257896423
Best feature set: ['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN', 'STEELLEVELINMOLD-SETMEAN', 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'LIQUIDUSTEMPMEAN', 'NARROWFACE1WATERFLOWMEAN', 'TUNDISHWEIGHTMEAN']
Best accuracy score: 0.7142857142857143


In [26]:
accuracy_store

[0.7142857142857143,
 0.5882352941176471,
 0.4369747899159664,
 0.5210084033613446,
 0.453781512605042,
 0.5126050420168067,
 0.6050420168067226,
 0.44537815126050423,
 0.6386554621848739,
 0.6134453781512605,
 0.5042016806722689,
 0.5798319327731093,
 0.5378151260504201,
 0.47058823529411764,
 0.47058823529411764,
 0.44537815126050423,
 0.5798319327731093,
 0.6218487394957983,
 0.5378151260504201,
 0.5714285714285714,
 0.4957983193277311,
 0.5966386554621849,
 0.4957983193277311,
 0.6050420168067226,
 0.5546218487394958,
 0.44537815126050423,
 0.5798319327731093,
 0.4369747899159664,
 0.6050420168067226,
 0.5546218487394958,
 0.453781512605042,
 0.5210084033613446,
 0.5462184873949579,
 0.44537815126050423,
 0.6050420168067226,
 0.6554621848739496,
 0.6050420168067226,
 0.4957983193277311,
 0.6050420168067226,
 0.5966386554621849,
 0.5042016806722689,
 0.6386554621848739,
 0.5210084033613446,
 0.47058823529411764,
 0.5714285714285714,
 0.5882352941176471,
 0.5882352941176471,
 0.51260

In [28]:
common_accuracy_store=accuracy_store
common_base_features=base_features
common_base_accuracy=base_accuracy
common_optimal_features=optimal_features

%store common_accuracy_store
%store common_base_features
%store common_base_accuracy
%store common_optimal_features

Stored 'common_accuracy_store' (list)
Stored 'common_base_features' (list)
Stored 'common_base_accuracy' (float64)
Stored 'common_optimal_features' (list)


In [29]:
len(new_train_X)

472

In [30]:
type(new_data)

pandas.core.frame.DataFrame